# CProject III 报告

指导老师：于仕琪

撰写人员：12112510 李涵

## 第一部分：项目简述（目录）

使用C编程语言设计一个函数来实现深度学习中的卷积神经网络。  
#### 建议的步骤：  
1. 理解卷积神经网络：请了解卷积运算、内核大小、填充和步幅。
2. 定义数据结构：应该为输入图像、内核和输出特征图定义数据结构。数据是三维的（宽度、高度和通道数量）。这个项目只需要`float32`。
3. 实现卷积运算：设计一个函数（当然也可以调用其他一些函数），它可以获取输入图像、一些内核和其他所需的参数。
4. 测试：测试功能，并检查结果是否正确。
5. 提高效率：提供一些统计数据来显示它在1x1、3x3和5x5内核上的效率。

在本次项目中，我编写了一个能够识别手写数字的卷积神经网络，并对它的卷积效率进行了优化。（[参考实现](https://blog.csdn.net/ShakalakaPHD/article/details/110694933))

## 第二部分：CNN流程的简述

在这一部分，我会先简要阐明一个CNN的实现流程（顺便概述CNN的常用概念）。[在此推荐一个通俗易懂的CNN原理教程](https://www.bilibili.com/video/av35087157/?vd_source=ebcb008f664a2fd98ddc00f701fcb081)

__1. 输入层：将数据集输入到卷积神经网络中。__
> __数据集（Dataset）__：在机器学习和数据挖掘中用于训练和评估模型的数据集合。数据集通常由一组样本组成，每个样本包含一个或多个特征和相应的标签或目标值。__典型案例__：[waymo数据集](https://waymo.com/open/)。waymo数据集包含来自传感器的高清晰度传感器数据、3D点云数据和定位数据等，以及在这些数据上进行的高质量标注，例如物体检测、跟踪、语义分割等。[数据集经典网站：imageNet](https://image-net.org/)

__2. 卷积层：将每一组的卷积核与原图的通道进行卷积。每组卷积核都会生成一个特征图（feature map）。__
> __卷积核（kernel）__：它是一个小的矩阵，通常是3x3或5x5的大小，它在输入图像上滑动，将每个位置上的像素值与卷积核中的值进行加权求和，得到输出特征图上对应位置的值。它的大小和形状通常根据任务需求进行调整。卷积核中的权重值是CNN模型需要学习的参数之一，通过 __反向传播算法（Backpropagation）__，CNN可以根据输入数据和目标输出来优化卷积核中的权重值，以提高模型的性能。  
>
> __步长（stride）__：在卷积核在输入图像上的滑动步长，通常情况下，步长越大，特征映射的大小就越小，计算量也就越小。但是，步长过大也可能导致信息的丢失，影响模型的性能。  
>
> __边界填充（padding）__：图像边缘的数据也至关重要。为了方便采集和提取特征，通常会在图像边缘添加一定量的填充像素，以扩大输入数据的尺寸。它能扩大卷积核的有效区域，保持特征图大小一致。__典型案例__：零填充（zero padding)。零填充在输入数据的边界周围添加0值像素，扩大输入数据的尺寸，以便卷积核能够沿着边界进行卷积操作。  
>
>__偏差（bias）__: 偏置是一个常数项，它被加到卷积操作的结果中，可以帮助模型更好地学习特征。偏置通常被初始化为零，然后由训练算法学习到最优值。偏置的作用是增加模型的灵活性，可以帮助模型更好地适应输入数据，从而提高模型的准确性。

__3. 激活函数：对卷积层的输出进行非线性变换，增加模型的非线性能力，便于后续的处理。__
> __典型案例__：Sigmoid函数和ReLU函数。Sigmoid函数将输入值压缩到$(0,1)$的范围内，可以用于二分类问题。其公式为：f(x) = $1 \over (1 + exp(-x))$。ReLU函数在x大于0时输出x，小于等于0时输出0。其公式为：$f(x) = max(0, x)$。

__4.池化层：对输入数据进行下采样操作，从而减少数据量、提高计算效率，并防止过拟合现象的发生。__
> __下采样（subsampling）__：即池化操作。用一个像素代替原图上邻近的若干像素。比如用一个区域内的最大像素作为该区域的代表。

__5.全连接层：将前面的卷积层或池化层输出的特征图扁平化为一维向量（每个连接都有一个权重值和一个偏置值），然后将其输入到全连接层中进行分类或回归任务。__
> __典型案例__：Softmax分类器和SVM分类器。Softmax分类器是一种常用的多分类算法，它的主要思想是将输入的数据通过一系列的线性变换和非线性变换后，映射到不同类别的概率分布上。它的训练过程通常采用交叉熵损失函数。SVM分类器是一种二分类算法，它的主要思想是通过构建一个最优的超平面来将不同类别的数据分隔开。它的训练过程通常采用最大间隔法，即选择使得不同类别之间的距离最大的超平面作为分类边界。

__6.输出层：输出并返回最终结果。__

__Q. CNN相对于全连接的优势？__  
这里顺便就为什么选择CNN而不是全部都用全连接来处理数据作个说明。
>__参数共享__：CNN的卷积层使用相同的卷积核对所有输入进行卷积操作，这意味着卷积层中的参数可以被多个神经元共享，从而大大减少了需要学习的参数数量。相比之下，全连接层的每个神经元都需要学习独立的权重和偏置，导致参数数量很大。
>
>__局部连接__：CNN的卷积操作仅针对输入数据中的局部区域进行，这意味着每个卷积核只学习一种特征，而不是对整个输入进行全局的学习。相比之下，全连接层对整个输入进行全局的学习，可能会导致过拟合。<font color=FF0000>也因为图像中，相邻像素有很强的相关性，而局部连接正好可以在捕捉相关性和减少参数量间达成微妙的平衡。</font>
>
>__位置不变性__：CNN的卷积操作对输入的平移不变，即无论输入图像的某个特征出现在哪个位置，其特征图都会出现在相同的位置。这种位置不变性使得CNN可以处理输入中的局部特征，而不受其位置影响。相比之下，全连接层无法处理位置不同的输入，需要对输入进行特殊的预处理。

* **测试环境：**

>CPU：AMD Ryzen 7 5800H（8核）    
>显卡：NVDIA GEFROCE RTX 3060 Laptop GPU 6GB  
>内存：16GB（3200MHz）  
>运行系统：Windows 10  
>C++编译器：mingw-GCC  
>C++版本：C++ 8.1.0  
>java版本：java 11.0.15  
>C++的IDE：Visual Studio Code 1.76.0  
>java的IDE：IntelliJ IDEA 2022.3(Ultimate Edition)

## 第三部分：正向传播代码的实现

基础框架的实现，我参考了Google Brain团队开发的TensorFlow机器学习框架。
因为C是面向过程的编程语言，所以没有类和相关的概念（如继承）。所以我只好使用结构嵌套的方式，实现伪继承（比如卷积层layer、池化层layer、全连接层layer，都包括`Layer`这样一个结构属性）。
~~我试了下，直接用一个大的`NetWork`结构，来储存所有的layer和传递过程中的feature_map，但最后发现在C中这样实现过于复杂，只好作罢。。。~~


### 1.读取数据集

我选择使用了Mnist数据集中的t10k，共含有一万张$28 * 28$的手写图片。  

我通过`read_mnist_data()`方法，实现了读取图像和标签数据的功能。它将数据集文件中的图像和标签数据读取到内存中，并保存到`ImageData`结构体数组中。`ImageData`结构体包含了图像的标签（代表该图像的数字是多少）和$28* 28$大小的灰度图像数据。该函数使用`fopen()`、`fread()`和`fclose()`等函数来读取文件。  

接着，我通过`shuffle_dataset()`方法，将读取到的数据集打乱，再用`create_batches()`方法，将这一万张图片分化为若干个大小相同的批次来处理。每一个批次都会进行一次完整的卷积操作。

__Q：为什么用一维数组存储图片的像素？__  
>A：为在多维数组中，每个元素占用的内存空间不一定是相邻的，而在一维数组中，所有元素都是相邻的，这有助于<font color=FF0000>提高内存访问的效率</font> 。在多维数组中，内存访问需要进行更多的计算，例如计算每个元素在内存中的位置，而在一维数组中，这些计算可以被避免。

__Q：为什么要在读取像素时多做一步处理，将它的数据范围限制在$(0,1)$之间？__
>A：两个原因。  
>
>一是能改善梯度消失问题（Vanishing Gradient Problem）。在神经网络中，梯度的大小是非常重要的，因为它决定了每个神经元的权重的更新量。如果梯度太小，那么神经元的权重就不会得到有效的更新，从而导致训练过程缓慢或者出现停滞现象。将数据范围限制在(0,1)之间可以避免梯度变得过小，因为输入数据的梯度也会被限制在同样的范围内，从而可以更好地进行权重更新。<font color=FF0000>这也和后续的梯度下降法（gradient descent）以最小化损失函数有关。</font> 
>
>二是方便进行数据预处理和归一化。显而易见，不详细分析了。

__Q：为什么使用了内置函数`builtin_bswap32()`?__
>A：它能将读取的大端字节序数据转换为小端字节序，这有利于保证程序在不同平台上的<font color=FF0000>兼容性</font>。

__Q：为什么要进行批处理，直接把一万张图片放进去跑不行吗？__
>A：是为了<font color=FF0000>优化计算效率和模型收敛速度</font>。在进行反向传播更新权重时，每个样本都需要计算一遍损失函数的梯度，然后根据梯度更新权重。如果每个样本都单独计算一遍，会增加计算量并且训练速度非常慢。而批处理将多个样本打包在一起，一次性进行计算，可以利用高效的并行计算加速计算过程。此外，批处理可以在一定程度上平滑训练过程，减少训练过程中的不稳定性，同时可以更好地更新模型参数，提高模型精度。

### 2.卷积层的实现

首先，我调用`initialize_conv_layer()`方法，对创建的`ConvLayer`卷积层结构体对象`conv_layer`进行初始化。需要注意的是，卷积层的`kernel`和`bias`的最完美精确的取值，应当通过反向传播算法来获得。~~在此，我先泛泛地分别初始化为0～0.1和0~~<font color="red"> 好吧事实证明不能这么简单，不优秀的卷积核初始化可能会引发梯度消失的问题（我就遇到了），在此我选择采用**He initialization**（何凯明等提出的一种鲁棒的神经网络参数（W）初始化方法，可以保证信息在前向传播和反向传播过程中能够有效流动，使不同层的输入信号的方差大致相等)，具体为：$stddev * ((float)rand() / (float)RAND_MAX * 2.0f - 1.0f)$ </font>。我还定义了一个枚举类`ActivationType`，里面有三种常见的激活函数的名称。我在`apply_activation()`方法中，通过枚举类的名称，返回对应激活函数的结果。 枚举类`PaddingType` 中包含了same和value两种方法，前一种是用$0$进行填充，后一种是不作更改。本次project的所有填充均采用same方法。  

接下来，通过调用`conv_forward()`方法，获取`train_data`的特征图。大致流程如下：以单个卷积核为例。卷积核从左上角开始扫描，与所有的输入通道的矩阵相乘。最后还要加上该输出通道对应的偏差值。大致的公式为（bias前面的计算步骤要重复ic次）：$$value = x_0w_0 + x_1w_1 + \cdots + x_nw_n + bias[oc]$$ 得到value后，将其放入激活函数中，做非线性化处理。然后，卷积核前进`stride`的步数，继续扫描，直到扫描完整张图片为止。最后便得到了该次卷积得到的`output_feature_map`。

__Q：怎么进行填充的？__
> A：对于'same'填充，目的是使输出特征图的空间尺寸与输入图像相同（便于跨层连接）。故而，当使用奇数大小的卷积核（例如$3 × 3$）时，需要在输入图像周围添加$(K - 1)\over 2$个像素的填充，其中K是卷积核的大小。因为我用一维数组来存储数据，故而处理过程会比较抽象，详情可参见`padding_matrix()`方法。

__Q：为什么随着单元的变大，卷积核的数量也变得越来越多？CNN不应该是浓缩数据吗？__
> A：这是因为在网络的前几层，卷积层通常用于捕捉较低级别的特征，如边缘和纹理。较少的卷积核足以捕捉这些低级特征。然而，在网络的后几层，我们希望捕捉更高级别的特征，如形状和对象部件，这需要更多的卷积核以捕捉更复杂和抽象的特征。另外，随着网络深度的增加，尽管卷积核的数量增加了，但由于池化层的存在，特征图的空间尺寸（宽度和高度）会逐渐减小。这样，在整个网络中，信息会被浓缩和抽象，同时在网络深层捕捉到更高级别的特征。

__Q：卷积核的内存空间应该是多大？__
> A：应当为`output_channels * input_channels * kernel_size * kernel_size`。这是因为在卷积层中，每个输出通道都与输入数据的所有通道进行卷积。因此，对于每个输出通道，我们需要一个卷积核与输入数据的每个通道进行卷积。这就导致了`output_channels`个卷积核组，每个卷积核组包含`input_channels`个卷积核，每个卷积核的大小为`kernel_size` * `kernel_size`。

### 3.池化层的实现

首先，类似的，我调用`initialize_pool_layer()`方法，对创建的`MaxPoolLayer`池化层结构体对象`max_pool_layer`进行初始化。然后滑动卷积核，将卷积核范围内的元素，以一个其中的最大值（本次project均采用 __Max Pooling__） 所替代（以$2×2$的卷积核为例，最终的特征图的该位置会输出一个最大值，而去除掉原本的四个元素）。池化层的输出特征图的尺寸的计算公式为：$output\_size = \lfloor \frac{input\_size - pool\_size}{stride} + 1 \rfloor$。

### 4.全连接层的实现

还是一样，调用`initialize_fully_connected_layer()`方法，对创建的`FullyConnectedLayer`全连接层结构体对象`fully_connected_layer`进行初始化。权重矩阵的大小为$weights\_size = input\_channels\_counts * kernel\_size * kernel \_size * output\_size$。  
和卷积层一样，权值和偏差的准确值也要通过反向传播算法所得，在此我先泛泛地初始化为$(-0.5,0.5)$。因为mnist数据集的最终目标是识别手写数字究竟是多少，所以我的`output_size`便直接初始化为$10$，表示$(0,9)$。在计算过程中，我遍历输入特征图的每个元素，并将元素值与对应的权重相乘，然后将结果累加到输出值上，最终加上偏差值。其公式为：$output[o] = \sum_{i=0}^{input\_size - 1} input[i] * weights[o * input\_size + i] + biases[o]$。  
最终，遍历这十个概率，取最大的概率所代表的数字为预测结果，并与标定的数据标签比较。如果相同，则本次正确，否则便错误。


## 第四部分：反向传播代码的实现

### 1.计算全连接层的误差梯度&&更新全连接层的权重矩阵

1. 首先共同使用softmax激活函数和交叉熵损失函数。首先使用softmax激活函数将神经网络的输出转换为概率分布（输出向量的每个元素都在0到1之间，并且它们的总和为1）。它的公式为：$\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{n} e^{x_j}}$  
2. 交叉熵损失函数衡量了两个概率分布（预测概率分布和目标概率分布）之间的差异。在使用softmax函数将网络输出转换为概率分布时，交叉熵损失函数可以直接比较这两个分布。根据其多分类问题的公式，我们有：$L(y, p) = -\sum_{i=1}^{C} y_i * log(p_i)$。其中$y_i$取值为$0$或$1$，表示标签i的真假，而$p_i$则表示第i个概率的值。在求出来该批次的所有损失后，我们求出其平均损失，其公式为：$L = \frac{1}{N} \sum_{j=1}^{N} L(y^{(j)}, p^{(j)})$。   
3. 在使用了交叉熵损失函数和softmax激活函数后，输出层的误差梯度计算得非常简单。对于第i个输出神经元，误差梯度为：$δ_i = p_i - y_i$。在实际运算中，只需把softmax激活函数的输出向量中，对应label的那一项减一即可。   
4. 接下来计算全连接层的反向传播梯度：$ input\_grad[i] = sum(output\_grad[j] * weights[j, i]) $，`i`为输入向量的每个元素，`j`从`0`取到`output_size - 1`。  
5. 接着利用计算好的梯度，去更新全连接层的权重矩阵，其主体算式如下：$weights[i, j] = weights[i, j] - learning\_rate * output\_grad[i] * input[j]$，其中`learning_rate`为学习率，`input`为输入的向量（即第四个单元的池化层输出的特征图）。

__Q：还有什么其他的损失函数？为什么最终会使用交叉熵损失函数？__  
> A：其他两种最为直接和常见的损失函数如下：  
 __Classification Error（分类错误率）：__$\text{Error Rate} = \frac{\text{Number of Misclassifications}}{\text{Total Number of Samples}}$  
 __Mean Squared Error (均方误差)：__$\text{MSE} = \frac{1}{N}\sum_{i=1}^{N}(y_i - \hat{y}_i)^2$  
> 
>第一种显而易见，过于简单粗暴，表现并不好。  
>
>第二种虽然表现良好，基本能正确判断模型的优秀情况，但如果我们使用sigmoid/softmax得到概率，配合MSE损失函数，采用梯度下降法进行学习时，会出现模型训练学习速率非常慢的情况。  
>
>而交叉熵损失函数可以避免梯度消失或梯度爆炸的问题，从而使训练更加稳定。同时，因为交叉熵损失函数能够提供更强的梯度信号，所以它能更快地进行收敛，这有助于优化算法更快地找到最优解。另外，交叉熵损失函数的梯度与预测值和目标值之间的差异成正比。这意味着，当网络预测值离目标值较远时，梯度较大，网络可以更快地进行调整；当网络预测值接近目标值时，梯度较小，网络的调整速度会减慢。

__Q：为什么要计算平均损失？__
> A：计算批次内所有样本的平均损失有助于更平滑地更新权重和偏差，从而减少训练过程中的噪声。使用平均损失还可以简化梯度的计算，因为我们只需要计算一次梯度，而不是为每个样本单独计算。

__Q：为什么误差梯度的计算公式中，并没有出现loss？__
> A：因为可以通过化简，将$∂L\over ∂𝑧_𝑘$从$∑𝑖(∂L / ∂𝑝i * ∂𝑝_𝑖 / ∂𝑧_𝑘)$最终转化为$-𝑦𝑘 + 𝑝𝑘$。事实上，`loss`只是一个体现，方便我们优化目标，监控训练过程和防止过拟合。

__Q：学习率是什么东东？为什么要设计它？它该怎么调整？__
> A：学习率（learning rate）是深度学习中的一个重要参数，它控制每次迭代中参数更新的步长，取值通常在0.001到0.1之间。如果学习率过大，会导致参数更新过快，可能会产生不稳定的振荡，甚至无法收敛；如果学习率过小，会导致参数更新缓慢，需要更长时间的训练才能达到理想的效果。 
常见调整方法如下：
>1. 经验设置：根据经验设置一个合适的学习率，然后观察模型在验证集上的表现，逐渐调整学习率的值，直到达到最优性能。
>2. 自适应学习率算法：如Adagrad、Adam、RMSprop等，这些算法可以根据梯度的大小来自动调整学习率的值，以实现更加高效和稳定的训练过程。
>3. 学习率衰减：在训练过程中逐渐降低学习率的值，以达到更好的收敛效果。  
>
>
>在本次项目中，我采用Adam算法，让学习率自适应“调整”（学习率实际上并没有变化，只是通过一些特殊的技巧，使其能不停地符合我们的需求）。  

__Q：Adam算法是啥？__
>Adam（Adaptive Moment Estimation）是一种自适应学习率的优化算法，它结合了Momentum和RMSProp的优点，旨在提高模型训练过程中的收敛速度和性能。
>
>**主要思想：**
>
>一阶矩估计（Momentum）：计算梯度的指数加权移动平均值，使得参数更新具有惯性效果，有助于加速收敛和减小震荡。
>
>二阶矩估计（RMSProp）：计算梯度平方的指数加权移动平均值，为每个参数自适应地调整学习率，从而降低梯度的尺度差异。
>
>**主要步骤：**
>
>初始化：给定一个初始学习率、一阶矩估计的衰减率（通常设为0.9）和二阶矩估计的衰减率（通常设为0.999），以及一个较小的常数（如1e-8）以防止除以零。同时，初始化一阶矩估计和二阶矩估计为零向量。
>
>在每次迭代中：
>
>a. 计算梯度：根据当前参数值和损失函数计算梯度。
>
>b. 更新一阶矩估计：将当前梯度的指数加权移动平均值与先前的一阶矩估计相结合。
>
>c. 更新二阶矩估计：将当前梯度平方的指数加权移动平均值与先前的二阶矩估计相结合。
>
>d. 对一阶矩估计和二阶矩估计进行偏差修正：因为初始值为零向量，所以需要进行偏差修正，以便更准确地估计一阶矩和二阶矩。
>
>e. 更新参数：结合修正后的一阶矩估计、二阶矩估计和学习率，按元素方式更新参数。
>
>重复步骤2，直至满足收敛条件或达到预设的最大迭代次数。  
>[附更详细的介绍](https://zhuanlan.zhihu.com/p/90169812)

### 2.将误差梯度经由池化层传播到卷积层。

池化层本身并不需要计算梯度和更新权重矩阵，但需要经过一些特殊的处理，才能把传递的梯度复原好，方便卷积层调用。  
因为我采用的是`max_pooling`的方法，所以我的处理过程如下：
1. 对于每个通道，遍历池化后的高度和宽度。对于每个池化区域，找到其中的最大值以及最大值所在的位置。
2. 将池化层输出数据的梯度传递给最大值所在的位置，其他位置全为默认值0。
3. 重复步骤2和3，直到遍历完所有的池化区域和通道。  

最后，`input_grad`代表池化层向卷积层反向传播的梯度。它的大小为$input\_channels * input\_size * input\_size$

### 3.卷积层的误差梯度传播&&卷积核的更新

前者在`accumulate_gradients()`方法中完成，后者在`update_conv_layer()`方法中完成。

详细原理和全连接层的更新类似，在此详细说说`grad_sum`的计算。这个梯度实际上就是损失函数关于卷积核的导数。根据链式法则，这个导数可以表示为损失函数关于当前层输出的导数（即`output_grad`）与输入特征图之间的卷积。grad_sum计算的过程就是在实现这个卷积操作。我们遍历每一个卷积核，并计算`input`与`output_grad`对应元素相乘的累加。

更新卷积核的`grad_sum`计算公式：  
$grad\_sum += input[ic * input\_size * input\_size + ih * input\_size + iw] * output\_grad[oc * output\_size * output\_size + oh * output\_size + ow]$

计算输入梯度的`grad_sum`计算公式：  
$grad\_sum += kernels[idx] * output\_grad[oc * output\_size * output\_size + oh * output\_size + ow]$  

对`bias`的更新和上述公式类似，便不再赘述。

__Q：有啥具体细节上的注意事项嘛？__
> A：注意到计算输入梯度时的`padded_ih`和`oh`。
>
>前者处理了填充问题。它在计算输入梯度时，将填充考虑在内。这样能有效计算正确的梯度位置。公式为：$padded\_ih = ih + padding - kh$
>
>后者处理了边界和步长问题。在计算输入梯度时，它根据步长计算了输出索引（oh和ow），并且只有当oh和ow在输出尺寸范围内时，才会将梯度累加到输入梯度中。这样做避免了访问无效的内存位置。公式为：$oh = \frac{padded\_ih}{stride}$

> 另外要分清楚`input_grad`和`kernel_grad_accum`的区别。前者存储了存储了每个输入通道上每个位置的梯度，所以大小为`input_channels * map_size * map_size`。后者的大小应为`output_channels * input_channels * kernel_size * kernel_size`，因为它存储了每个输出通道和输入通道之间每个卷积核元素的梯度累积。

在反向传播到最开始的卷积层后，相当于一轮完整的训练就结束了，接下来便开始第二轮，依次循环，直到达到指定次数。

__Q：是每一张图片的每一轮都要更新一次嘛？还是说一个大batch训练完一轮后再更新？__
> A：我在训练完一个批次的数据后更新参数。因为使用一个批次的数据可以更好地估计梯度，并且可以提高训练过程的稳定性。因此，在训练完一批图片之后再更新参数（包括卷积层的权重、偏置以及批量归一化的参数）是更好的选择。此外，我采取的是介于随机梯度下降（每训练一张图片就更新一次参数）和批量梯度下降（每个批次更新一次参数）之间的方法：小批量梯度下降（批次更新，但`batch_size`较小）。这样既可以减小噪声，又能在计算资源允许的范围内实现较快的收敛速度。  
>
>实际表现为，我先统计每一张图片的每一层的累积梯度。然后在全部图片过完一轮后，先将累计梯度矩阵除以`batch_size`，再以此为根据，去更新每一层的权重矩阵。

## 第五部分：代码的优化&加速

在本节中，我会阐明我的代码的优化方法。其中有一些优化措施已经在前面详细介绍了（比如ReLU激活函数，Adam优化算法等），下面便不再赘述。

### 1.批量归一化

**为什么要使用：**

在训练过程中，每层输入的分布不断的变化，这使得下一层需要不断的去适应新的数据分布，在深度神经网络中，这让训练变得非常复杂而且缓慢。对于这样，往往需要设置更小的学习率、更严格的参数初始化。通过使用批量归一化(Batch Normalization, BN)，在模型的训练过程中利用小批量的均值和方差调整神经网络中间的输出，从而使得各层之间的输出都符合均值、方差相同高斯分布，这样的话会使得数据更加稳定，无论隐藏层的参数如何变化，可以确定的是前一层网络输出数据的均值、方差是已知的、固定的，这样就解决了数据分布不断改变带来的训练缓慢、小学习率等问题。

**定义：**  

其涉及到的参数如下：

**bn_gamma：** 批量归一化中的缩放因子。在应用批量归一化后，激活值会被缩放。`bn_gamma`是一个可学习的参数，用于调整缩放大小。

**bn_beta：** 批量归一化中的偏移因子。在应用批量归一化后，激活值会被平移。`bn_beta`是一个可学习的参数，用于调整平移量。

**bn_mean：** 批量归一化中的均值。它是在训练过程中，对一批数据计算每个通道的均值。`bn_mean`用于将激活值减去它们的均值，使得激活值具有零均值。这有助于控制激活值的分布，使得训练更加稳定。

**bn_variance：** 批量归一化中的方差。它是在训练过程中，对一批数据计算每个通道的方差。`bn_variance`用于将激活值除以它们的标准差，使得激活值具有单位方差。这同样有助于控制激活值的分布，使得训练更加稳定。

批量归一化通常在卷积层或全连接层之后、激活函数之前应用。这样可以使得网络在更深层次上保持稳定的分布，从而提高训练速度和性能。

**具体实现：**  
其具体用处体现在`conv_forward()`方法中。在激活函数前，它对value进行了如下操作（其中eps为极小值）：

$$ value = \frac{value - mean[oc]}{sqrtf(variance[oc] + eps)} （使其具有零均值和单位方差） $$
$$ value = gamma[oc] * value + beta[oc]  （控制每个通道的输出值的范围和中心位置）$$  
它们的更新操作，和卷积核/权重矩阵的累积梯度更新很类似，体现在`conv_accumulate_gradients()`和`update_conv_layer()`方法中，也是采用了Adam算法来自适应调整学习率来更新，其中计算批量归一化参数梯度累积的各类参数的公式为;
$$idx = oc * size * size + oh * size + ow$$
  $$ normalized\_output = (output[idx] - conv\_layer->bn\_mean[oc]) / sqrtf(conv\_layer->bn\_variance[oc] + eps)（标准化输出）$$
   $$gamma\_grad\_sum += output\_grad[idx] * normalized\_output（对γ更新） $$
    $$ beta\_grad\_sum += output\_grad[idx]（对β更新）$$
    $$mean\_grad\_sum -= output\_grad[idx] * conv\_layer->bn\_gamma[oc] * normalized_input（对平均值更新）$$
    
$$variance\_grad\_sum -= \frac{output\_grad[idx] * conv\_layer->bn\_gamma[oc] * normalized\_input * (output[idx] - conv\_layer->bn\_mean[oc])}{(2.0f * conv\_layer->bn\_variance[oc] * sqrtf(conv\_layer->bn\_variance[oc] + eps))}（对方差更新）$$

（囿于本人能力和时间所限，仅对卷积层进行批量归一化操作） 

【注意：每一个变量的移动平均值`m`和移动方差`v`都是唯一的。我直接在`ConvLayer`结构体中，把相关的属性加进去了。】

[详细介绍](https://zhuanlan.zhihu.com/p/81190407)

### 2.并行化计算

**定义：**（熟客了）

在并行计算中，任务被分割为多个子任务，每个子任务可以独立地执行。这些子任务可以在多个处理器或计算机核心上同时执行，从而并行处理大量的数据。在并行计算中，每个处理器或计算机核心都有自己的内存，并且可以独立地访问和修改数据。

其优点是可以显著提高计算速度和效率，特别是在大规模数据处理和计算密集型应用中。同时，通过将任务分配到多个处理器或计算机核心上，可以减少单个处理器或计算机核心的负载，延长硬件寿命。

本次项目中，我使用**OpenMP库**进行并行化计算。

**具体实现：**

eg：详见某些`for`循环的上头。

**<font color="red">警告：</font>**

在某些情况下，使用并行化代码可能会导致段错误（Segmentation Fault）。以下是一些可能的原因：

1. 数据竞争（Data race）：多个线程可能同时访问和修改同一个内存位置，导致不一致的状态。为避免数据竞争，请确保正确同步线程并在必要时使用临界区、锁或原子操作。

2. 堆栈溢出（Stack overflow）：OpenMP会为每个线程创建一个独立的堆栈。如果线程数很多，或者每个线程使用了大量的堆栈内存，这可能会导致堆栈溢出。在这种情况下，可以尝试减少线程数或减小每个线程所需的堆栈内存。

3. 错误共享（False sharing）：当两个或多个线程频繁访问同一个缓存行（cache line）中的不同变量时，可能会导致性能下降和段错误。为避免错误共享，请确保线程访问的数据在内存中尽可能分开。

4. 动态内存分配问题：如果在并行区域内动态分配内存，需要确保内存分配和释放是线程安全的。使用线程安全的内存分配函数（如 calloc、malloc 和 free 的线程安全版本）可以避免这个问题。

体现在我的项目中的话，使用`collapse(4)`来将小循环嵌套为大循环时，很有可能会因为变量之间的依赖关系而导致段错误。这时可以采用`atomic`来进行原子化操作，使得同一内存位在同一时间最多只会有一个线程去修改和访问。

### 3.快速傅里叶变换

**（这位更是重量级！！）**

首先，我们先来看看离散傅里叶变换（DFT）。

**定义：**

离散傅里叶变换（Discrete Fourier Transform，DFT）是一种数学变换。DFT将时域上的离散信号转换到频域上，可以将一个复杂的周期信号分解成一系列正弦波的叠加，这样可以更直观地观察信号的频率成分和特征。DFT可以用于音频、图像等领域的频率分析和滤波、编码等。

而IDFT则将频域上的信号恢复到时域上，可以将经过DFT变换后的信号重新还原成原始信号。IDFT可以用于信号重构和滤波器设计等。

**原理：**

鉴于其完整的推导过程过于复杂（相对于我而言），我在此用通俗易懂的语言来解释。

具体来说，对于给定的两个长度为$N$的向量`a`和`b`，在此举例为 $A = [1, 2, 3, 4]$，$B = [5, 6, 7, 8]$。它们的卷积运算（多项式乘积）可以通过以下两个步骤完成：

1.将`a`和`b`补零为长度为$2N$的向量。  
$A' = [1, 2, 3, 4, 0, 0, 0, 0]$  
$B' = [5, 6, 7, 8, 0, 0, 0, 0]$  

2.对补零后的向量执行DFT变换，即将`a`和`b`转换为频域上的两个向量，然后将它们逐元素相乘。  
$A\_freq = [10+0i, -2+2i, -2+0i, -2-2i, 0+0i, 0+0i, 0+0i, 0+0i]$  
$B\_freq = [26+0i, -2-2i, -2+0i, -2+2i, 0+0i, 0+0i, 0+0i, 0+0i]$  
→  
$C\_freq = [26+0i, -8-8i, -6+0i, -8+8i, 0+0i, 0+0i, 0+0i, 0+0i]$

（应用公式为：$X_k = \sum\limits_{n=0}^{N-1} x_n e^{-2\pi i kn/N}$）

3.对结果执行IDFT变换，即将频域上的向量转换回时域，得到卷积运算的结果。  
$C = [5, 16, 34, 60, 61, 52, 32，0，0]$  

（应用公式为：$x_n=\frac{1}{N}\sum_{k=0}^{N-1}X_ke^{2\pi i kn/N}$）

DFT的时间复杂度为$O(N^2)$（因为DFT算法中需要对每个输出元素进行$N$次求和，每次求和需要计算$N$个输入元素和对应的旋转因子）

<font color="red">但是这显然不太够，这时就轮到我们的**FFT**了!!</font>

**定义：**

FFT，全称为快速傅里叶变换（Fast Fourier Transform），是一种计算离散傅里叶变换（DFT）的高效算法。算法的本质是将长度为 $N$ 的 DFT 分解为多个长度为 $\frac{N}{2}$ 的 DFT，递归地实现 DFT 的计算，从而达到快速计算 DFT 的目的。FFT 算法的时间复杂度是 $O(NlogN)$，相比普通的 DFT 算法的时间复杂度 $O(N^2)$ 要高效得多，尤其在 N 较大时表现更加明显。 (**divide and conquer knowledge!**)

**具体实现：**

因为时间原因（和实力原因），我选择直接调用库（FFTW ( the Faster Fourier Transform in the West) ，一个快速计算离散傅里叶变换的标准C语言程序集，可计算一维或多维实和复数据以及任意规模的DFT，且运行速度比Eigen和opencv自带的FFT库函数快10倍以上。）来实现。[安装流程详见](https://blog.csdn.net/Jhon_ranble/article/details/120576590)

1. 调用`padding_matrix()`和`pad_kernel()`，让卷积核的尺寸等同于填充完的填充后的输入特征图的尺寸。

2. 调用`FFTW`库的`fftw_plan_dft_r2c_2d()`方法，创建若干个计划对象，将实数数据的“二”维矩阵转换为复数形式的离散傅里叶变换（感谢强大的`FFTW`库，只要我正确填写了宽和高的参数，就算我的数据是一维的，它也还是能正常处理）。然后调用` fftw_execute()`方法来实际执行FFT，并将结果存储在若干个`fftw_complex`类型的变量里。

3. 在频域内进行复数乘法，计算公式为：$(a+bi)(c+di)=(ac-bd)+(ad+bc)i$

4. 进行IDFT逆转换，获得正确的输出特征图。

**<font color="red">警告：</font>**

因为我们在计算过程中，需要两个向量的长度相同，所以需要对卷积核进行零填充（如上文的`pad_kernel()`方法所示）。并且，最终的时间复杂度会为$O(N^2 * log(N^2))$，其中$N$为输入特征图的尺寸。而直接进行卷积运算的时间复杂度为$O(N^2 * M^2)$，其中$M$为卷积核的尺寸。倘若卷积核尺寸很小，即$M < 2logN$，那么FFT其实并没有起到优化效果！！！（在本次项目就是这样的。。。）所以，我们又需要借助其他的手段来优化。。！

### 5.Winograd卷积算法

相较于FFT，Winograd卷积算法在处理小卷积核时有更大的优势。（在此我用`conv_forward_combined()`方法，针对不同的卷积核的尺寸，来选择使用不同的优化算法）

**定义：**

Winograd卷积是一种快速卷积算法，它通过减少浮点乘法的数量来加速卷积计算。Winograd卷积的核心思想基于Winograd最小乘法算法，该算法允许使用比标准卷积方法更少的乘法操作来计算卷积。尽管这种方法需要额外的加法操作，但由于乘法操作相对较慢，因此可以在计算性能受限的场景中实现更高的计算效率。

**原理：**

Winograd卷积的原理基于将输入数据、滤波器（卷积核）和输出数据变换到另一个域（通常称为Winograd域）。在这个新的域中，卷积操作可以通过更少的乘法计算来完成，同时仍然保留了原始卷积的结果。将数据转换回原始域后，可以得到最终的卷积结果。

Winograd卷积算法的关键步骤如下：

1. 对输入数据和滤波器进行变换，将它们映射到Winograd域。
2. 在Winograd域中执行乘法操作，这些操作比在原始域中执行的乘法操作少。
3. 将结果从Winograd域变换回原始域，得到最终的卷积结果。

**作用：**

Winograd卷积的主要作用是加速卷积计算。由于卷积计算在许多计算密集型应用中（例如深度学习和图像处理）都起着关键作用，因此使用Winograd卷积可以显著提高这些应用的性能。特别是在计算能力有限的设备（如嵌入式设备和移动设备）上，Winograd卷积提供了一种高效的卷积计算方法。然而，这种加速方法并非没有代价，由于需要额外的数据变换，因此Winograd卷积可能会引入一些计算误差。在实践中，这些误差通常足够小，不会对算法的性能产生显著影响。

**具体使用：**

1. 首先下载falcon库（Fast Linear CONvolution library，一个高效的线性卷积库）：[官网链接](https://falconframework.org/)（试图git clone但显示我没权限。。）

2. 运行以下命令来配置和构建系统并安装库：  
`cmake -G "Visual Studio 16 2019" -A x64 ..`  
`cmake --build . --config Release --target install`  

3. 在`.vscode`文件夹中的`c_cpp_properties.json`文件，配置好Falcon库的`include`目录的实际路径。（或者在cMakeLists.txt里面配置好）

4. 使用`#include <falcon/convolution/winograd_convolution.h>`来引用。

5. 详见`conv_forward_winograd()`方法中`winograd_conv`变量的一系列操作。（黑箱封装，具体原理很难说明。。）

### 6.SIMD并行计算

**定义：**（其实也是熟客了）

SIMD (Single Instruction Multiple Data) 是一种并行计算的技术，通过将单一指令应用于多个数据元素，同时在单个时钟周期内处理多个数据元素，从而提高了程序的执行效率。在计算机体系结构中，SIMD 可以通过向量寄存器、SIMD指令集和特定硬件实现。使用 SIMD 技术，可以将多个元素一次性加载到寄存器中，并同时执行加法操作，从而减少访问内存的次数，提高了效率。

**<font color="red">警告：</font>**

因为我的特征图的尺寸较小，所以很多SIMD执行单元大概率会处于空闲状态，这反而会浪费了SIMD加速的潜力。另外，我的程序中的循环嵌套较多，而每一层嵌套中的数据量其实较小，所以很难利用好SIMD并行性。综上所述，此次项目，我并未利用SIMD进行优化。

### 7. O3编译优化

**定义：**（熟客+1）

-O3是GCC和Clang编译器的优化选项之一，表示对代码进行最高级别的优化，以提高程序的运行速度和性能。

**实现方式：**  

-O3优化级别的实现方式包括以下几个方面：

函数内联（Function inlining）：
函数内联是一种编译器优化技术，它将函数调用替换为函数的实际代码，减少了函数调用的开销，从而提高了程序的运行速度。

循环展开（Loop unrolling）：
循环展开是一种将循环展开成多个重复执行的语句块的优化技术。它可以减少循环控制开销和循环变量的计算，从而提高程序的运行速度。

向量化（Vectorization）：
向量化是一种通过使用SIMD指令（Single Instruction Multiple Data）将多个数据处理操作同时进行的技术。它可以将一些串行的数据操作并行化，从而提高程序的运行速度。

代码重排（Code reordering）：
代码重排是一种将代码重新排列以提高缓存利用率和指令流水线效率的技术。它可以将代码中的热点数据和指令移到缓存的前面，从而减少缓存的读取时间，提高程序的运行速度。

消除冗余代码（Dead code elimination）：
消除冗余代码是一种将代码中不必要的计算或条件分支删除的技术。它可以减少程序的执行次数，从而提高程序的运行速度。

优化算法（Algorithm optimization）：
优化算法是一种通过更改算法以提高程序效率的技术。它可以在不改变程序功能的前提下，减少算法执行时间，从而提高程序的运行速度。

**实际实现：**

在`cMakeLists.txt`中的`CMAKE_C_FLAGS`里面，添加`-O3`即可。

## 第六部分：实战效果

首先我想介绍下VSCode的debug功能怎么用。

1. 安装CodeLLDB插件。并创建好build文件夹用来放置配置和生成文件。

2. 在.vscode文件夹中，创建并编写两个文件launch.json（用来设置debug选项，包括指示生成的可执行文件，可执行文件的输入参数，类型等）和tasks.json（辅助launch.json运行）

3. 在vscode的“运行和调试”处，点击运行，选择gdb编译器，打好断点即可。

4. 倘若报错`ERROR: Unable to start debugging. Unexpected LLDB output from command "-exec-run".`，则可能是lldb的环境变量配置产生了冲突，需手动调整下。

5. （当然如果嫌麻烦，可以直接printf()了事）




然后我贴几张实际跑出来的效果图（最终敲定的是，总共五个batch，每个batch二十张图片，训练二十轮）：![](https://s2.loli.net/2023/04/22/kNV58EMtGF6cjLI.png)
![](https://s2.loli.net/2023/04/22/hI8e9mkbTlLAcg7.png)
![](https://s2.loli.net/2023/04/22/yUiY245f8HPzeo1.png)

对比于网上其他水友跑出来的结果（他们的正确率能轻松破95%，loss最终收敛也比我低。。），感觉我的中间还是有大问题的。。（估计是梯度优化算法哪里没写好，考虑的情况不够周全。或者是还有很多方法没能实现，导致一些特殊情况没能处理好。。而且我的训练次数也不够多。。囿于时间因素，我也没法再修改了。。/(ㄒoㄒ)/~）

接下来再来张粗略地分析速度变化的柱状图：![](https://s2.loli.net/2023/04/22/yIXctUWK69vONjm.png)

可以看到，O3编译优化就已经极大地提升程序效率了，其他的功能，我认为要不就是O3优化已经帮我们实现了，要不就是我的处理并不是特别到位，所以提升效率并不明显。（比如FFT其实就不适合我这次的CNN）

## 第七部分：收获&不足

**收获：**  
1. 系统而完整地认识了最基础的卷积神经网络的框架。
2. 了解了一些著名的数据集，以及数据集的读取和处理方式。
3. 有条不紊地搭建一个大型的框架，更加完善地进行了代码的分配。
4. 学习了如何在vscode上debug。
5. 认识了一些激活函数以及它们的衍生型，知道了它们如何避免梯度消失的原理。
6. 了解了一些如何避免梯度爆炸的方法（采用批量归一化，Adam算法）。
7. 对傅里叶变换有了初步的了解（如从DFT到FFT的演化过程）
8. and many many....

**不足：**  
1. 仍然是之前两次的通病：其实很多方法的很多部分是相同的，完全可以合并到一个方法里面。（其实我这样做主要是为了可读性233）。
2. 未能有机会，在其他操作平台/开发板上进行性能对比测试（实在是受设备所限）。
3. 兼容性这一块只是浅浅涉猎，未能贯彻全文。
4. 还有一些精妙的卷积设计，比如[Network Pruning](https://arxiv.org/pdf/1605.06431.pdf)，能以微小的精确度损失为代价，极大地减少运算和内存的消耗。囿于时间所限，无法实践，深表遗憾。
5. 对网络规模的调整（通常而言，加深加宽）应当可以使其更容易训练。水平有限，无法更深一步探讨。
6. 最后的准确率显然不过关。。。路漫漫其修远兮，吾将上下而求索。。

## 第八部分：感想&&参考文献

从零开始搭建一个卷积神经网络是真的难！！！！！！  
这个项目花了我约莫八十个小时了！！！！ 
新的概念太多了，甚至因为一开始不太清楚具体结构，整体框架推翻重来了好几次！！！  
虽然的确是学到了很多东西，但真的好折磨aaaaaaaaaaaaaaaaa  
卷积神经网络涉及到的东西真的太多太多了。。。感觉根本学不完。。。
/(ㄒoㄒ)/~~

**参考文献：**
* [gemm矩阵乘法优化](https://zhuanlan.zhihu.com/p/66958390)  
* [卷积神经网络 C++ 从零开始实现](https://zhuanlan.zhihu.com/p/468100301)  
* [视频详解CNN](https://www.bilibili.com/video/BV1gA4y1S7tb?p=2&spm_id_from=pageDriver&vd_source=ebcb008f664a2fd98ddc00f701fcb081)  
* [那么……什么是卷积？](https://www.bilibili.com/video/BV1Vd4y1e7pj/?share_source=copy_web&vd_source=ce9050ced2aed7b18646473277d9c09c)  
* [交叉熵函数详解](https://www.zhihu.com/tardis/zm/art/35709485?source_id=1005)  
* [反向传播](https://zhuanlan.zhihu.com/p/115571464)  


**特别致谢：** 
* 万能的至高无上的主：chatgpt 4.0 version